In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install wandb

     |████████████████████████████████| 1.8MB 6.3MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 174kB 30.2MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=16f77a99a2aa6404c4d31f80a9e96b83ac82c5b9f3ade83ea8733d2f62d64fca
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=8a34e1987ca08780c817988b76eff60637cfa392cedd90f0bd73359a5987e952
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import exp
from math import log
numpy_rng = np.random.RandomState(1234)

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/Fashion_MNIST/fashion-mnist_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Fashion_MNIST/fashion-mnist_test.csv')

In [7]:
train_data_df = train_df.drop('label', axis=1)
train_data_np = train_data_df.to_numpy()
train_bin = np.where(train_data_np > 127, 1, 0)
train_data = train_bin.reshape(train_bin.shape[0], 784)
train_labels = train_df['label'].to_frame().to_numpy()
train_data,val_data,train_labels,val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)
#train_data = [train_data[i].flatten() for i in range(train_data.shape[0])]
#train_data = [np.reshape(train_data[i],(1,train_data[i].shape[0])) for i in range(len(train_data))]

In [8]:
train_data.shape

(54000, 784)

In [9]:
test_data_df = test_df.drop('label', axis=1)
test_data_np = test_data_df.to_numpy()
test_bin = np.where(test_data_np > 127, 1, 0)
test_data = test_bin.reshape(test_bin.shape[0], 784)
test_labels = test_df['label'].to_frame().to_numpy()
#test_data = [test_data[i].flatten() for i in range(test_data.shape[0])]
#test_data = [np.reshape(test_data[i],(1,test_data[i].shape[0])) for i in range(len(test_data))]

In [10]:
def eval(data,W,b):
  return softmax(np.reshape(np.matmul(W.T,data),(10,1)) + b)

In [11]:
def one_hot_encoding(l, L):
  import numpy
  e = []
  for i in range(L):
    if i == l:
      e.append(1)
    else:
      e.append(0)
  e_y = np.asarray(e)
  return np.reshape(e_y,(10,1))

In [12]:
def softmax(vector):
	e = exp(vector)
	return e / e.sum()

In [13]:
def train(data,labels,W,b,lr = 0.05):

  t = []
  o = []
  for j in range(data.shape[0]):
    o.append(eval(data[j],W,b))
    t.append(one_hot_encoding(labels[j],10))
  for i in range(10):
    grad_w = np.zeros([W.shape[0],W.shape[1]])
    grad_b = np.zeros([b.shape[0],1])
    for j in range(data.shape[0]):
      mul2 = np.reshape((o[j]-t[j]),[10,1])
      mul1 = np.reshape(data[j],[32,1])
      grad_w += np.matmul(mul1,mul2.T)
      grad_b += mul2
    W = W - lr*grad_w
    b = b - lr*grad_b
  
  return W,b

In [14]:
def test(data,labels,W,b):
  cross_entropy = 0;
  count = 0;
  t = []
  o = []
  for j in range(data.shape[0]):
    o.append(eval(data[j],W,b))
    t.append(one_hot_encoding(labels[j],10))
    index_t = np.argmax(t[j],axis = 0)
    index_o = np.argmax(o[j],axis = 0)
    if index_t == index_o:
      count += 1
    cross_entropy -= log(o[j][index_t]+1e-300)
  accuracy = count/data.shape[0]

  return cross_entropy, accuracy

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
def train_test(x_train, y_train, x_test,y_test):
  logisticRegr = LogisticRegression(max_iter = 1000,multi_class='multinomial')
  logisticRegr.fit(x_train, y_train)
  #print(logisticRegr.classes_)
  y_pred = logisticRegr.predict(x_test)
  pred = logisticRegr.predict_proba(x_test)
  #print(y_pred)
  #print(y_test)
  loss = log_loss(y_test,pred)
  accuracy = accuracy_score(y_test, y_pred)
  return loss, accuracy

In [26]:
import sys
import numpy

def sigmoid(x):
    return 1. / (1 + np.exp(-x))

numpy.seterr(all='ignore')

class RBM(object):
     def __init__(self, input=None, n_visible=2, n_hidden=3, W=None, hbias=None, vbias=None):
        
        if W is None:
            initial_W = np.random.randn(n_visible, n_hidden) * 0.01
            W = initial_W

        if hbias is None:
            hbias = np.random.randn(n_hidden) * 0.01  # initialize h bias 0

        if vbias is None:
            vbias = np.random.randn(n_visible) * 0.01 # initialize v bias 0

        self.n_visible = n_visible 
        self.n_hidden = n_hidden   
        self.input = input
        self.W = W
        self.hbias = hbias
        self.vbias = vbias

def gibbs_sampling(rbm, lr=0.1, k=200, r=10):

  ph_mean, ph_sample = sample_h_given_v(rbm,np.random.randn(rbm.input.shape[0], rbm.input.shape[1]))

  chain_start = ph_sample

  for step in range(k):
      print("Sampling k={}".format(step))
      if step == 0:
          nv_means, nv_samples,\
          nh_means, nh_samples = gibbs_hvh(rbm,chain_start)
      else:
          nv_means, nv_samples,\
          nh_means, nh_samples = gibbs_hvh(rbm,nh_samples)
  
  dW_2 = numpy.zeros((rbm.n_visible,rbm.n_hidden))
  dvbias_2 = numpy.zeros(rbm.n_visible)
  dhbias_2 = numpy.zeros(rbm.n_hidden)

  for step in range(r):
    print("Sampling r={}".format(step))
    nv_means, nv_samples,\
    nh_means, nh_samples = gibbs_hvh(rbm,nh_samples)
    dW_2 += numpy.dot(nv_samples.T, nh_means)
    dvbias_2 += numpy.sum(nv_samples, axis=0)
    dhbias_2 += numpy.sum(nh_means, axis=0)

  # chain_end = nv_samples
  rbm.W += lr * (numpy.dot(rbm.input.T, ph_mean)
                  - dW_2)
  rbm.vbias += lr * numpy.mean(rbm.input - dvbias_2, axis=0)
  rbm.hbias += lr * numpy.mean(ph_sample - dhbias_2, axis=0)

def sample_h_given_v(rbm, v0_sample):
    h1_mean = propup(rbm,v0_sample)
    h1_sample = numpy_rng.binomial(size=h1_mean.shape, n=1, p=h1_mean)
    return [h1_mean, h1_sample]


def sample_v_given_h(rbm, h0_sample):
    v1_mean = propdown(rbm,h0_sample)
    v1_sample = numpy_rng.binomial(size=v1_mean.shape, n=1, p=v1_mean) 
    return [v1_mean, v1_sample]

def propup(rbm, v):
    pre_sigmoid_activation = np.dot(v, rbm.W) + rbm.hbias
    return sigmoid(pre_sigmoid_activation)

def propdown(rbm, h):
    pre_sigmoid_activation = np.dot(h, rbm.W.T) + rbm.vbias
    return sigmoid(pre_sigmoid_activation)


def gibbs_hvh(rbm, h0_sample):

    v1_mean, v1_sample = sample_v_given_h(rbm,h0_sample)
    h1_mean, h1_sample = sample_h_given_v(rbm,v1_sample)

    return [v1_mean, v1_sample,
            h1_mean, h1_sample]

def get_hidden_reps(rbm, v,t):
    h_v = sigmoid(np.dot(v, rbm.W) + rbm.hbias)
    h_t = sigmoid(np.dot(t, rbm.W) + rbm.hbias)
    #reconstructed_v = sigmoid(np.dot(h, rbm.W.T) + rbm.vbias)
    return h_v,h_t
        

def train_rbm(rbm, v,t,learning_rate=0.05, k = 200, r=10):
    gibbs_sampling(rbm,lr=learning_rate,k=k,r=r)
    return get_hidden_reps(rbm,v,t)

In [27]:
def swp():
  hyperparameter_defaults = dict(
    n_hidden=64,
    k = 200,
    r = 10
  )

  wandb.init(project="Assignment-4", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_h_layers_{}_k_{}_r- Gibbs".format(config.n_hidden, config.k,config.r)
  n_visible=784
  rbm = RBM(input=train_data, n_visible=n_visible, n_hidden=config.n_hidden)

  for epoch in range(5):

    hidden_rep,hidden_rep_test= train_rbm(rbm,val_data,test_data,k = config.k,r=config.r)
    print('Training epoch: %d' %epoch)
    
    #W_l, b_l = train(hidden_rep,val_labels,W_l,b_l,lr=0.1)
    loss, accuracy = train_test(hidden_rep, val_labels, hidden_rep_test,test_labels)
    print('Loss:{} Accuracy:{}'.format(loss,accuracy))
    metrics = {'epoch':epoch, 'accuracy': accuracy, 'loss': loss}
    wandb.log(metrics)
  wandb.run.finish()

In [28]:
sweep_config = {
  "name": "Sweep - Gibbs",
  "method": "grid",
  "project": "Assignment-4",
  "metric":{
      "name":"accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "n_hidden": {
            "values":[64,128,256]
        },
        "k": {
            "values":[200,300]
        },
        "r": {
            "values":[10,20,30]
        },
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 2iyg666u
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/2iyg666u


In [ ]:
wandb.agent('2iyg666u', swp)